# Import necessary libraries

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',100)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import scipy.stats as stats

from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
import xgboost as xg
from sklearn.ensemble import GradientBoostingRegressor

from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score

# Hyperparameter tuner and Cross Validation
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

import shap

#sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set(rc={"figure.dpi":300, 'savefig.dpi':800})

import matplotlib.font_manager as font_manager
font = font_manager.FontProperties(family='Times New Roman')
plt.rcParams['font.family'] = 'Times New Roman'

In [2]:
df = pd.read_csv("df1.csv")

In [3]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per"]]
columns = []
for i in X.columns:
    columns.append(i)

# Voting Regressor - Standard Scaling

In [12]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per"]]

scaler = StandardScaler()
kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

n=0
r2 = []
rmse = []
mae=[]

for trn_idx, test_idx in kf.split(df[columns],y['drug_perm_per']):
    print(f"fold: {n+1}")
    X_tr,X_tst = df[columns].iloc[trn_idx], df[columns].iloc[test_idx]
    y_tr,y_tst = y['drug_perm_per'].iloc[trn_idx], y['drug_perm_per'].iloc[test_idx]
    
    model_1 = xg.XGBRegressor(max_depth = 10, eta = 0.2, verbosity=0)
    model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
    model_4 = GradientBoostingRegressor()
    model_5 = neighbors.KNeighborsRegressor(n_neighbors = 1)
    final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])
    
    nmr = ['drug_load', 'drug_mw', 'mn_length', 'mn_surface_area', 'permeation_time']
    for i in columns:
        if i in nmr:
            X_tr[i] = scaler.fit_transform(X_tr[i].values.reshape(-1,1))
            X_tst[i] = scaler.transform(X_tst[i].values.reshape(-1,1))
    
    final_model.fit(X_tr, y_tr)
    y_pred = final_model.predict(X_tst)
    print("R2: ", r2_score(y_tst, y_pred))
    rmse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=False)
    print(": model's RMSE = {}".format(rmse1))
    rmse.append(rmse1)
    mae1 = mean_absolute_error(y_tst, final_model.predict(X_tst))
    print(": model's MAE = {}".format(mae1))
    mae.append(mae1)
    r2.append(r2_score(y_tst, y_pred))
    print("------------------------------------------------------------------------------------------------------")
    
    n+=1
    
print("R2: {}, RMSE: {}, MAE: {}".format(np.mean(r2), np.mean(rmse), np.mean(mae)))

fold: 1
R2:  0.9865682926960642
: model's RMSE = 2.64130273231211
: model's MAE = 1.6948923485495584
------------------------------------------------------------------------------------------------------
fold: 2
R2:  0.9871022236259701
: model's RMSE = 3.099035795966184
: model's MAE = 1.9939350898132047
------------------------------------------------------------------------------------------------------
fold: 3
R2:  0.9875682209110335
: model's RMSE = 2.4080879401116047
: model's MAE = 1.3842176306189335
------------------------------------------------------------------------------------------------------
fold: 4
R2:  0.9663612956978012
: model's RMSE = 4.806661164074709
: model's MAE = 2.5883087592135525
------------------------------------------------------------------------------------------------------
fold: 5
R2:  0.9805757375418345
: model's RMSE = 2.522630810490951
: model's MAE = 1.360665228865997
-------------------------------------------------------------------------------

# Voting Regressor - MinMax Scaling

In [11]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per"]]

scaler = MinMaxScaler()
kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

n=0
r2 = []
rmse = []
mae = []

for trn_idx, test_idx in kf.split(df[columns],y['drug_perm_per']):
    print(f"fold: {n+1}")
    X_tr,X_tst = df[columns].iloc[trn_idx], df[columns].iloc[test_idx]
    y_tr,y_tst = y['drug_perm_per'].iloc[trn_idx], y['drug_perm_per'].iloc[test_idx]
    
    model_1 = xg.XGBRegressor(max_depth = 10, eta = 0.2, verbosity=0)
    model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
    model_4 = GradientBoostingRegressor()
    model_5 = neighbors.KNeighborsRegressor(n_neighbors = 1)
    final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])
    
    for i in columns:
        X_tr[i] = scaler.fit_transform(X_tr[i].values.reshape(-1,1))
        X_tst[i] = scaler.transform(X_tst[i].values.reshape(-1,1))
    
    final_model.fit(X_tr, y_tr)
    y_pred = final_model.predict(X_tst)
    print("R2: ", r2_score(y_tst, y_pred))
    rmse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=False)
    print(": model's RMSE = {}".format(rmse1))
    rmse.append(rmse1)
    mae1 = mean_absolute_error(y_tst, final_model.predict(X_tst))
    print(": model's MAE = {}".format(mae1))
    mae.append(mae1)
    r2.append(r2_score(y_tst, y_pred))
    print("------------------------------------------------------------------------------------------------------")
    
    n+=1
    
print("R2: {}, RMSE: {}, MAE: {}".format(np.mean(r2), np.mean(rmse), np.mean(mae)))

fold: 1
R2:  0.9865316170238093
: model's RMSE = 2.6449063514804143
: model's MAE = 1.6985450831605335
------------------------------------------------------------------------------------------------------
fold: 2
R2:  0.9869486858739296
: model's RMSE = 3.1174270008026648
: model's MAE = 1.989785999322971
------------------------------------------------------------------------------------------------------
fold: 3
R2:  0.9871356274038154
: model's RMSE = 2.449627256888964
: model's MAE = 1.373069744076518
------------------------------------------------------------------------------------------------------
fold: 4
R2:  0.9660546685346834
: model's RMSE = 4.828518567211665
: model's MAE = 2.6108544282218857
------------------------------------------------------------------------------------------------------
fold: 5
R2:  0.9803481570637516
: model's RMSE = 2.53736572565646
: model's MAE = 1.4221655824844184
-------------------------------------------------------------------------------

# Voting Regressor - No Scaling

In [7]:
kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
columns = ['drug_name', 'drug_load', 'drug_mw', 'mn_surface_area', 'mn_length', 'skin_type', 'mn_type', 'permeation_time']
n=0
r2 = []
rmse = []
mae = []
mse=[]

for trn_idx, test_idx in kf.split(df[columns],y['drug_perm_per']):
    print(f"fold: {n+1}")
    X_tr,X_tst = df[columns].iloc[trn_idx], df[columns].iloc[test_idx]
    y_tr,y_tst = y['drug_perm_per'].iloc[trn_idx], y['drug_perm_per'].iloc[test_idx]
    
    model_1 = xg.XGBRegressor()
    model_2 = RandomForestRegressor(random_state=1)
    model_4 = GradientBoostingRegressor()
    model_5 = neighbors.KNeighborsRegressor()
    final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])
    
    final_model.fit(X_tr, y_tr)
    y_pred = final_model.predict(X_tst)
    print("R2: ", r2_score(y_tst, y_pred))
    rmse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=False)
    print(": model's RMSE = {}".format(rmse1))
    rmse.append(rmse1)
    mse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=True)
    print(": model's MSE = {}".format(mse1))
    mse.append(mse1)
    mae1 = mean_absolute_error(y_tst, final_model.predict(X_tst))
    print(": model's MAE = {}".format(mae1))
    mae.append(mae1)
    r2.append(r2_score(y_tst, y_pred))
    print("------------------------------------------------------------------------------------------------------")
    
    n+=1
    
print("R2: {}, RMSE: {}, MAE: {}, MSE: {}".format(np.mean(r2), np.mean(rmse), np.mean(mae), np.mean(mse)))

fold: 1
R2:  0.9872431153642658
: model's RMSE = 2.574096861032304
: model's RMSE = 6.62597464997636
: model's MAE = 1.6894889353976683
------------------------------------------------------------------------------------------------------
fold: 2
R2:  0.9824493895849541
: model's RMSE = 3.6150581229217202
: model's RMSE = 13.06864523210231
: model's MAE = 2.306882855912823
------------------------------------------------------------------------------------------------------
fold: 3
R2:  0.9874590508405326
: model's RMSE = 2.4186381800196384
: model's RMSE = 5.849810645848709
: model's MAE = 1.4013410632844563
------------------------------------------------------------------------------------------------------
fold: 4
R2:  0.9388622189493837
: model's RMSE = 6.480055343257742
: model's RMSE = 41.99111725168322
: model's MAE = 3.378893732138194
------------------------------------------------------------------------------------------------------
fold: 5
R2:  0.9693167955677099
: model's

# Hyperparameter tuned models

In [4]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per"]]

model_1 = xg.XGBRegressor(max_depth = 10, eta = 0.2, verbosity = 0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
model_4 = GradientBoostingRegressor()
model_5 = neighbors.KNeighborsRegressor(n_neighbors = 1)
final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(final_model, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
#print(sorted(scores.tolist()))
print("RMSE score:", np.negative(np.mean(scores)))

RMSE score: 3.234851343319971


In [7]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per"]]

model_1 = xg.XGBRegressor(max_depth = 10, eta = 0.2, verbosity = 0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
model_4 = GradientBoostingRegressor()
model_5 = neighbors.KNeighborsRegressor(n_neighbors = 1)
final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(final_model, X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
#print(sorted(scores.tolist()))
print("R2 score:", (np.mean(scores)))

R2 score: 0.9757427371009746


# Conversion

## Not Tuned conversion

In [9]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per","drug_perm_amt"]]

kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
columns = ['drug_name', 'drug_load', 'drug_mw', 'mn_surface_area', 'mn_length', 'skin_type', 'mn_type', 'permeation_time']
n=0
rmse = []
mse = []
mae = []
r2 = []
myrmse = []
mymse = []
mymae = []
myr2 = []

for trn_idx, test_idx in kf.split(df[columns],y):
    print(f"fold: {n+1}")
    X_tr,X_tst = df[columns].iloc[trn_idx], df[columns].iloc[test_idx]
    y_tr,y_tst = y['drug_perm_per'].iloc[trn_idx], y['drug_perm_per'].iloc[test_idx]
    yt1 = y['drug_perm_amt'].iloc[test_idx]
    
    model_1 = xg.XGBRegressor()
    model_2 = RandomForestRegressor(random_state=1)
    model_4 = GradientBoostingRegressor()
    model_5 = neighbors.KNeighborsRegressor()
    final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])
    
    final_model.fit(X_tr, y_tr)
    y_pred = final_model.predict(X_tst)
    
    m = []
    for i in range(len(test_idx)):
        m.append((y_pred[i]*X_tst["drug_load"].values[i])/100)
    m = np.array(m)
    
    rmse1 = mean_squared_error(yt1, m, squared=False)
    print(": My RMSE = {}".format(rmse1))
    myrmse.append(rmse1)
    
    mse1 = mean_squared_error(yt1, m, squared=True)
    print(": My MSE = {}".format(mse1))
    mymse.append(mse1)
    
    mae1 = mean_absolute_error(yt1, m)
    print(": My MAE = {}".format(mae1))
    mymae.append(mae1)
    
    r21 = r2_score(yt1, m)
    print(": My R2 = {}".format(r21))
    myr2.append(r21)
    
    
    
    rmse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=False)
    print(": model's RMSE = {}".format(rmse1))
    rmse.append(rmse1)
    
    mse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=True)
    print(": model's MSE = {}".format(mse1))
    mse.append(mse1)
    
    mae1 = mean_absolute_error(y_tst, final_model.predict(X_tst))
    print(": model's MAE = {}".format(mae1))
    mae.append(mae1)
    
    r21 = r2_score(y_tst, final_model.predict(X_tst))
    print(": model's R2 = {}".format(r21))
    r2.append(r21)
    
    print("------------------------------------------------------------------------------------------------------")
    
    n+=1
    
print("My RMSE: {}, My MSE: {}, My MAE: {}, My R2: {}, model's RMSE: {}, model's MSE: {}, model's MAE: {}, model's R2: {}".format(np.mean(myrmse), np.mean(mymse), np.mean(mymae), np.mean(myr2), np.mean(rmse), np.mean(mse), np.mean(mae), np.mean(r2)))

fold: 1
: My RMSE = 384.5255050567761
: My MSE = 147859.86403916875
: My MAE = 196.6884313220733
: My R2 = 0.9700572383049589
: model's RMSE = 2.5740968610323036
: model's MSE = 6.625974649976359
: model's MAE = 1.6894889353976679
: model's R2 = 0.9872431153642658
------------------------------------------------------------------------------------------------------
fold: 2
: My RMSE = 881.4111569777097
: My MSE = 776885.6276447846
: My MAE = 452.31356044893215
: My R2 = 0.9835593292714914
: model's RMSE = 3.6150581229217202
: model's MSE = 13.06864523210231
: model's MAE = 2.3068828559128227
: model's R2 = 0.9824493895849541
------------------------------------------------------------------------------------------------------
fold: 3
: My RMSE = 257.4200212363197
: My MSE = 66265.06733330729
: My MAE = 162.7267270421961
: My R2 = 0.9815077824883175
: model's RMSE = 2.4186381800196384
: model's MSE = 5.8498106458487085
: model's MAE = 1.4013410632844563
: model's R2 = 0.9874590508405326

: My RMSE = 1023.1312587943694
: My MSE = 1046797.572722151
: My MAE = 544.0769785599128
: My R2 = 0.931972657086763
: model's RMSE = 3.9862014045276823
: model's MSE = 15.889801637458469
: model's MAE = 2.6756542952230595
: model's R2 = 0.9723021097202214
------------------------------------------------------------------------------------------------------
fold: 25
: My RMSE = 632.6863624934598
: My MSE = 400292.03328520554
: My MAE = 391.07887888044877
: My R2 = 0.9750180956139137
: model's RMSE = 1.5073722398911322
: model's MSE = 2.272171069594409
: model's MAE = 1.1780628653562515
: model's R2 = 0.9882002037604437
------------------------------------------------------------------------------------------------------
fold: 26
: My RMSE = 417.1463526766323
: My MSE = 174011.0795514173
: My MAE = 263.3086764568645
: My R2 = 0.9738478875428691
: model's RMSE = 3.376687136563517
: model's MSE = 11.402016018233525
: model's MAE = 2.1027189338616155
: model's R2 = 0.9858540872970891
-----

- 'My ___' RMSE/R2 values indicate the permeation amount evaluation, which was calculated as: (percentage_predicted*drug_load)/100).
- 'model's ___' values indicate the percentage_predicted evaluation

## Tuned conversion

In [15]:
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per","drug_perm_amt"]]

kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
columns = ['drug_name', 'drug_load', 'drug_mw', 'mn_surface_area', 'mn_length', 'skin_type', 'mn_type', 'permeation_time']
n=0
rmse = []
mse = []
mae = []
r2 = []
myrmse = []
mymse = []
mymae = []
myr2 = []

for trn_idx, test_idx in kf.split(df[columns],y):
    print(f"fold: {n+1}")
    X_tr,X_tst = df[columns].iloc[trn_idx], df[columns].iloc[test_idx]
    y_tr,y_tst = y['drug_perm_per'].iloc[trn_idx], y['drug_perm_per'].iloc[test_idx]
    yt1 = y['drug_perm_amt'].iloc[test_idx]
    
    model_1 = xg.XGBRegressor(max_depth = 10, eta = 0.2, verbosity=0)
    model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
    model_4 = GradientBoostingRegressor()
    model_5 = neighbors.KNeighborsRegressor(n_neighbors = 1)
    final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])
    
    final_model.fit(X_tr, y_tr)
    y_pred = final_model.predict(X_tst)
    
    m = []
    for i in range(len(test_idx)):
        m.append((y_pred[i]*X_tst["drug_load"].values[i])/100)
    m = np.array(m)
    
    rmse1 = mean_squared_error(yt1, m, squared=False)
    print(": My RMSE = {}".format(rmse1))
    myrmse.append(rmse1)
    
    mse1 = mean_squared_error(yt1, m, squared=True)
    print(": My MSE = {}".format(mse1))
    mymse.append(mse1)
    
    mae1 = mean_absolute_error(yt1, m)
    print(": My MAE = {}".format(mae1))
    mymae.append(mae1)
    
    r21 = r2_score(yt1, m)
    print(": My R2 = {}".format(r21))
    myr2.append(r21)
    
    
    
    rmse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=False)
    print(": model's RMSE = {}".format(rmse1))
    rmse.append(rmse1)
    
    mse1 = mean_squared_error(y_tst, final_model.predict(X_tst), squared=True)
    print(": model's MSE = {}".format(mse1))
    mse.append(mse1)
    
    mae1 = mean_absolute_error(y_tst, final_model.predict(X_tst))
    print(": model's MAE = {}".format(mae1))
    mae.append(mae1)
    
    r21 = r2_score(y_tst, final_model.predict(X_tst))
    print(": model's R2 = {}".format(r21))
    r2.append(r21)
    
    print("------------------------------------------------------------------------------------------------------")
    
    n+=1
    
print("My RMSE: {}, My MSE: {}, My MAE: {}, My R2: {}, model's RMSE: {}, model's MSE: {}, model's MAE: {}, model's R2: {}".format(np.mean(myrmse), np.mean(mymse), np.mean(mymae), np.mean(myr2), np.mean(rmse), np.mean(mse), np.mean(mae), np.mean(r2)))

fold: 1
: My RMSE = 420.87656081176533
: My MSE = 177137.0794407396
: My MAE = 207.7022632050864
: My R2 = 0.9641283766117587
: model's RMSE = 1.4826499689980013
: model's MSE = 2.1982509305697744
: model's MAE = 1.1613749342706163
: model's R2 = 0.9957677421054163
------------------------------------------------------------------------------------------------------
fold: 2
: My RMSE = 1002.6365402393243
: My MSE = 1005280.0318230822
: My MAE = 486.81751093975475
: My R2 = 0.9787259830726269
: model's RMSE = 3.0580479178877575
: model's MSE = 9.35165706809765
: model's MAE = 1.7921302847074345
: model's R2 = 0.9874411396879821
------------------------------------------------------------------------------------------------------
fold: 3
: My RMSE = 266.9984003184214
: My MSE = 71288.145772596
: My MAE = 152.0909701013336
: My R2 = 0.9801060204013592
: model's RMSE = 2.789613214121534
: model's MSE = 7.781941884401476
: model's MAE = 1.5772815482730698
: model's R2 = 0.9833169065731273
-

: My RMSE = 655.3774602224255
: My MSE = 429519.61536759697
: My MAE = 334.60570236856364
: My R2 = 0.9720871743267514
: model's RMSE = 1.4644095220008138
: model's MSE = 2.1444952481266517
: model's MAE = 1.1408964553585792
: model's R2 = 0.9962618794467456
------------------------------------------------------------------------------------------------------
fold: 25
: My RMSE = 710.9737199580836
: My MSE = 505483.63047103555
: My MAE = 406.297547549995
: My R2 = 0.9684531724962864
: model's RMSE = 1.7666904179571792
: model's MSE = 3.1211950329017126
: model's MAE = 1.2487723537582356
: model's R2 = 0.9837910684168998
------------------------------------------------------------------------------------------------------
fold: 26
: My RMSE = 434.7085473336886
: My MSE = 188971.5211249658
: My MAE = 257.723898399251
: My R2 = 0.9715994838696755
: model's RMSE = 2.795692255587363
: model's MSE = 7.815895187951159
: model's MAE = 1.64358685426476
: model's R2 = 0.9903032085863542
--------

# Graph actual vs prediction

In [86]:
# Per - VR
X = df.drop(['drug_perm_per','drug_perm_amt'], axis=1)
y = df[["drug_perm_per"]]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, shuffle=True)

model_1 = xg.XGBRegressor(max_depth = 10, eta = 0.2, verbosity = 0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
model_4 = GradientBoostingRegressor()
model_5 = neighbors.KNeighborsRegressor(n_neighbors = 1)
final_model = VotingRegressor(estimators=[('xgb', model_1), ('rf', model_2), ('gbr', model_4), ('knn',model_5)])

final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('RMSE:', rmse)
print('R2:', r2)

RMSE: 2.8500801679877044
R2: 0.9817036177207833


In [90]:
y_test.to_csv('y_test.csv')

In [91]:
pd.DataFrame({'y_pred':list(y_pred)}).to_csv('y_pred.csv')